In [ ]:
import glob
import imageio
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tqdm
import albumentations as A
import csv
ffs = glob.glob('C:/Gdrive_tudo/code/project/spell_training/*.png')
def get_label(f):
    if 'Incendio' in f:
        label =1
    elif 'aqua' in f:
        label =2
    elif 'leviosa' in f:
        label =3
    elif 'arresto' in f:
        label =4
    elif 'alohomora' in f:
        label =5
    elif 'lumos' in f:
        label = 6

    return label
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_list = []
for ii in tqdm.tqdm(range(len(ffs))):
    f = ffs[ii]
    #im = imageio.imread(f)
    print(im.shape, f)
    label = get_label(f)
    #im_d = torch.from_numpy(im[None, ...][None, ...]).to(device).float()/255
    #label_d = torch.from_numpy(np.array([label])).to(device)
    train_list.append([im, label])

import os
from PIL import Image



def gray_image(input_folder, output_folder, size):
    # 檢查輸出資料夾是否存在，如果不存在則創建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍歷輸入資料夾中的所有檔案
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 調整影像大小為 (size, size)
            image = Image.open(input_path)
            resized_image = image.resize((size, size))

            # 將調整大小後的影像進行灰階轉換
            gray_image = resized_image.convert('L')
            gray_image.save(output_path)

# 使用範例
input_folder = 'C:/Gdrive_tudo/code/project/spell_training1'  # 輸入資料夾的路徑
output_folder = 'C:/Gdrive_tudo/code/project/spell_training'  # 輸出資料夾的路徑
size = 28  # 目標大小


gray_image(input_folder, output_folder, size)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        #x = self.fc1(x.view(x.size(0), -1))
        x = self.relu3(x)
        x = self.fc2(x)
        return x


# Initialize the model
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)



epochs = 20000
csv_filename = 'training2_results.csv'
header = ['Epoch', 'Loss', 'Accuracy', 'Val Loss', 'Val Accuracy']


with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    # Training loop
    for epoch in range(epochs):
        loss_sum = 0
        count = 0

        # Training
        model.train()
        for ii in range(len(train_list)):
            im, label = train_list[ii]
            transform = A.Compose([
                A.ShiftScaleRotate(p=0.5),
                A.OpticalDistortion(p=0.5),
                A.GridDistortion(p=0.5),
            ])
            im = transform(image=im)['image']

            im_d = torch.from_numpy(im[None, ...][None, ...]).to(device).float() / 255
            label_d = torch.from_numpy(np.array([label])).to(device)

            output_d = model(im_d)
            loss = criterion(output_d, label_d.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            count += 1

        # Compute and print average training loss
        avg_loss = loss_sum / count
        print(epoch, avg_loss)

        # Save training results to the CSV file
        row = [epoch + 1, avg_loss, 0, 0, 0]  # Placeholder for validation results
        writer.writerow(row)

# Save the trained model
torch.save(model, 'void.pt')
print("Saving the model as void.pt")


In [ ]:
import glob
import imageio
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tqdm
import albumentations as A
import csv
import os
from PIL import Image

# Glob the image files(讀取圖片檔案路徑)
ffs = glob.glob('C:/Gdrive_tudo/code/project/spell_training/*.png')

def get_label(f):
    # 根據檔案名稱獲取標籤
    if 'Incendio' in f:
        label = 1
    elif 'aqua' in f:
        label = 2
    elif 'leviosa' in f:
        label = 3
    elif 'arresto' in f:
        label = 4
    elif 'alohomora' in f:
        label = 5
    elif 'lumos' in f:
        label = 6

    return label

# Set device (CPU or GPU)(設定裝置（使用 CPU 或 GPU）)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare the training data(準備訓練資料)
train_list = []
for ii in tqdm.tqdm(range(len(ffs))):
    f = ffs[ii]
    im = imageio.imread(f)
    print(im.shape, f)
    label = get_label(f)
    train_list.append([im, label])

#Adjust input images(調整輸入影像)

def gray_image(input_folder, output_folder, size):
    # Check if the output folder exists, if not, create it(檢查輸出資料夾是否存在，如果不存在則創建)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all the files in the input folder(翻找輸入資料夾中的所有檔案)
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Resize the image to (size, size)
            # 調整影像大小為 (size, size)
            image = Image.open(input_path)
            resized_image = image.resize((size, size))

            # Convert the resized image to grayscale
            # 將調整大小後的影像進行灰階轉換
            gray_image = resized_image.convert('L')
            gray_image.save(output_path)

input_folder = 'C:/Gdrive_tudo/code/project/spell_training1'  # Input Path(輸入資料夾的路徑)
output_folder = 'C:/Gdrive_tudo/code/project/spell_training'  # Output Path(輸出資料夾的路徑)
size = 28  # Size(目標大小)

gray_image(input_folder, output_folder, size)

# Define the CNN model(定義 CNN 模型)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        #x = self.fc1(x.view(x.size(0), -1))
        x = self.relu3(x)
        x = self.fc2(x)
        return x

# Initialize the model(初始化模型)
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 1000
csv_filename = 'training_results.csv'
header = ['Epoch', 'Loss', 'Accuracy', 'Val Loss', 'Val Accuracy']

with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    # Training loop
    # 訓練迴圈
    for epoch in range(epochs):
        loss_sum = 0
        count = 0

        # Training
        # 訓練
        model.train()
        for ii in range(len(train_list)):
            im, label = train_list[ii]
            transform = A.Compose([
                A.ShiftScaleRotate(p=0.5),
                A.OpticalDistortion(p=0.5),
                A.GridDistortion(p=0.5),
            ])
            im = transform(image=im)['image']

            im_d = torch.from_numpy(im[None, ...][None, ...]).to(device).float() / 255
            label_d = torch.from_numpy(np.array([label])).to(device)

            output_d = model(im_d)
            loss = criterion(output_d, label_d.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            count += 1

        # Compute and print average training loss(計算並打印平均訓練損失)
        avg_loss = loss_sum / count
        print(epoch, avg_loss)

        # Save training results to the CSV file(將訓練結果保存到 CSV 檔案中)
        row = [epoch + 1, avg_loss, 0, 0, 0]  # Placeholder for validation results
        writer.writerow(row)

# Save the trained model(儲存已訓練的模型)
torch.save(model, 'quark.pt')
print("Saving the model as quark.pt")


 38%|█████████████████████████████▍                                                | 175/464 [00:00<00:00, 1746.12it/s]

(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora1.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora10.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora11.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora12.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora13.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora135.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora136.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora137.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora138.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora139.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora14.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora140.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora141.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\alohomora142.png
(28, 28) C:/Gdrive_tudo/code/project/spel

100%|██████████████████████████████████████████████████████████████████████████████| 464/464 [00:00<00:00, 1741.34it/s]


(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa58.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa59.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa6.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa60.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa61.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa62.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa63.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa64.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa65.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa66.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa67.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa68.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa69.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa7.png
(28, 28) C:/Gdrive_tudo/code/project/spell_training\leviosa70.png
(28, 28) C:/

0 1.723253901421053
1 1.442669609465264
2 1.2626471703996438
3 1.1198988473662008
4 1.0144775216063975
5 1.1612876000262142
6 1.203866350996687
7 1.2596822315706582
8 0.9522777596011953
9 1.1096829130658132
10 1.0635101577951493
11 0.945800425448432
12 0.8748846754909083
13 0.8780548884152245
14 1.000097104664872
15 1.1709937966354496
16 0.9401043454323219
17 0.8407108168147217
18 1.1356012354973937
19 0.7800795222418275
20 1.096806654256316
21 0.9767660349367568
22 1.018644555207288
23 1.784954591282277
24 1.0988515660922624
25 1.2488227009247344
26 0.8792018217302249
27 0.8509803956200359
28 1.1480240893144542
29 1.0299872438960753
30 0.7202967127794709
31 0.8411325961076535
32 0.7851043650397777
33 0.8329381361125446
34 0.8964599207439256
35 0.9581314123717992
36 1.0037978967534296
37 0.794369633832471
38 0.9914900051542984
39 0.8429782192983237
40 0.6481849301047752
41 0.8346229498149405
42 0.6163880475122168
43 0.584395988325715
44 0.6097535411484062
45 0.79607931080128
46 0.75877

355 0.159192817274503
356 0.15511331643871026
357 0.1442803166798101
358 0.18692053167322972
359 0.16868654318127396
360 0.20783788326009192
361 0.12625944279932708
362 0.16895927413942147
363 0.1905448681555234
364 0.1520966678246519
365 0.20505584469027555
366 0.14153020670920954
367 0.13622660488530988
368 0.10284145436988432
369 0.17225474113627776
370 0.17356590512844616
371 0.12466168852629206
372 0.18318605183364553
373 0.15037244295301905
374 0.12864556157193172
375 0.21582134175999207
376 0.17308028133454428
377 0.15750033625814105
378 0.1125219277613053
379 0.14484574048248733
380 0.18115455313082818
381 0.11227291969436444
382 0.12131111470118362
383 0.14773975314577023
384 0.15966410488596244
385 0.1356770762194412
386 0.11307155977632422
387 0.1564450486842869
388 0.14307148105251924
389 0.13535581022100257
390 0.1462319629189696
391 0.1093652067069469
392 0.10895146704226202
393 0.1637170459817143
394 0.12585267263131028
395 0.1844828371159594
396 0.16375803143603532
397 

700 0.12709000778679186
701 0.06012534592944513
702 0.10902672058714699
703 0.11065535328814467
704 0.060841596730173225
705 0.13457719593557116
706 0.09563337188073283
707 0.13118063273046052
708 0.12231200111751885
709 0.1159183298731896
710 0.07746706644632208
711 0.0974716568712901
712 0.09884130376762537
713 0.1727367937775673
714 0.0888391332552224
715 0.15332369927455655
716 0.07102459466977515
717 0.1194509592001964
718 0.08694836421441039
719 0.10444043170402657
720 0.11091407026656916
721 0.11486817831861822
722 0.08799461260418621
723 0.11209597403289795
724 0.09265958028419435
725 0.0659546828214588
726 0.0725615731882154
727 0.1497445246763605
728 0.058290094671878886
729 0.084754448961952
730 0.10325127870160994
731 0.12492277563001926
732 0.11050528231112712
733 0.0986545643419923
734 0.10769675006794745
735 0.09396576892205044
736 0.07774524747092573
737 0.2513239209497911
738 0.07256524079894332
739 0.09510543871251177
740 0.06259687289817804
741 0.038930476423885856
7